In [ ]:
#cài đặt thư viện
!pip install -q lightning

In [ ]:
#download dataset
#!wget https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv
!gdown 1aXs9anuFLEOO2iQ9mUYDOkni04v_Fy9J

In [ ]:
import lightning as L
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from torchmetrics import Accuracy
from sklearn.preprocessing import StandardScaler

In [ ]:
#read dataset
data = pd.read_csv("/content/iris.csv")
data.head()

Iris là dataset nhận dạng loại hoa từ 4 features bao gồm sepal.length, sepal.widt, petal.length, và petal.width

In [ ]:
#encode cho label
labels = {}
for index, element in enumerate(data["variety"].unique()):
  labels[element] = index

data.loc[:, "labels"] = data["variety"].apply(lambda x: labels[x])
data.head()

In [ ]:
# Tạo class Dataset để đọc dữ liệu
COLUMNS = data.drop(['variety', "labels"], axis=1).columns.to_list()  # Lấy danh sách các cột không bao gồm 'variety' và 'labels'

class DataSet(torch.utils.data.Dataset):
    def __init__(self, data, normalizer, columns=COLUMNS):
        super(DataSet, self).__init__()  # Khởi tạo lớp cha
        self.data = data  # Lưu trữ dữ liệu đầu vào
        self.feature = normalizer.transform(self.data[columns].values)  # Chuẩn hóa các đặc trưng
        self.feature = torch.tensor(self.feature).float()  # Chuyển đổi các đặc trưng thành tensor kiểu float
        self.label = torch.tensor(self.data["labels"].values)  # Chuyển đổi nhãn thành tensor

    def __len__(self):
        return len(self.feature)  # Trả về số lượng mẫu trong dataset

    def __getitem__(self, idx):
        # Trả về một từ điển chứa đặc trưng và nhãn tại chỉ số idx
        return {"feature": self.feature[idx],
                "label": self.label[idx]}


In [ ]:
# Chia và chuẩn hóa dataset

BATCH_SIZE = 128  # Kích thước batch cho DataLoader
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% - 20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
# Chia tập huấn luyện thành tập kiểm tra và tập val (60% train, 20% test 20% val)
test_data, val_data = train_test_split(train_data, test_size=0.25, random_state=0)

# Khởi tạo StandardScaler để chuẩn hóa dữ liệu
normalizer = StandardScaler()
# Tính toán các tham số chuẩn hóa dựa trên tập huấn luyện
normalizer.fit(train_data[COLUMNS].values)

# Tạo DataLoader cho tập huấn luyện, cho phép trộn và sử dụng nhiều workers
train_loader = torch.utils.data.DataLoader(DataSet(train_data, normalizer), batch_size=BATCH_SIZE,
                                           shuffle=True, num_workers=2)

# Tạo DataLoader cho tập val
val_loader = torch.utils.data.DataLoader(DataSet(val_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

# Tạo DataLoader cho tập kiểm tra
test_loader = torch.utils.data.DataLoader(DataSet(test_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

# Lấy một batch dữ liệu từ train_loader
data_loader = next(iter(train_loader))
# In ra kích thước của các đặc trưng trong batch
print(len(data_loader["feature"]))
# In ra kích thước của các nhãn trong batch
print(len(data_loader["label"]))
# In ra các đặc trưng và nhãn (bình luận lại để không in ra)
#print(data_loader["feature"], "\n", data_loader["label"])


In [ ]:
# Tạo model
class Model(L.LightningModule):
    def __init__(self, num_classes=len(labels), learning_rate=5e-3, input_dim=len(COLUMNS)):
        super(Model, self).__init__()  # Khởi tạo lớp cha
        self.learning_rate = learning_rate  # Lưu trữ learning rate
        # Tạo mô hình MLP với các lớp tuyến tính và hàm kích hoạt ReLU
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),  # Lớp đầu vào với kích thước input_dim và đầu ra 64
            torch.nn.ReLU(),                  # Hàm kích hoạt ReLU
            torch.nn.Linear(64, 32),          # Lớp tiếp theo với đầu vào 64 và đầu ra 32
            torch.nn.ReLU(),                  # Hàm kích hoạt ReLU
            torch.nn.Linear(32, num_classes), # Lớp đầu ra với đầu vào 32 và đầu ra số lớp
        )

    def forward(self, x):
        return self.mlp(x)  # Thực hiện phép biến đổi thông qua mô hình MLP

    def training_step(self, batch, batch_idx):
        x = batch["feature"]  # Lấy đặc trưng từ batch
        y = batch["label"]    # Lấy nhãn từ batch
        y_pred = self(x)      # Dự đoán nhãn bằng mô hình
        # Tính toán loss bằng hàm CrossEntropy
        loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        self.log('train_loss', loss, prog_bar=True)  # Ghi lại loss huấn luyện để hiển thị

        # Tính độ chính xác
        acc = (y_pred.argmax(dim=1) == y).float().mean()  # Tính accuracy
        self.log('train_acc', acc, prog_bar=True)  # Ghi lại độ chính xác huấn luyện

        return loss # Trả về loss

    def validation_step(self, batch, batch_idx):
        x = batch["feature"]  # Lấy đặc trưng từ batch
        y = batch["label"]    # Lấy nhãn từ batch
        y_pred = self(x)      # Dự đoán nhãn bằng mô hình
        # Tính toán loss cho tập val
        val_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        self.log('val_loss', val_loss, prog_bar=True)  # Ghi lại loss val để hiển thị
        # Tính độ chính xác
        val_acc = (y_pred.argmax(dim=1) == y).float().mean()  # Tính accuracy
        self.log('val_acc', val_acc, prog_bar=True)  # Ghi lại độ chính xác huấn luyện

        return val_loss # Trả về loss val

    def configure_optimizers(self):
        # Khởi tạo optimizer Adam với learning rate đã định
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer  # Trả về optimizer


In [ ]:
# Train model
model = Model()  # Khởi tạo một thể hiện của mô hình

# Tạo Trainer từ PyTorch Lightning với số epoch tối đa là 100 và kích hoạt tính năng phát hiện bất thường
trainer = L.Trainer(max_epochs=100, detect_anomaly=True)

# Bắt đầu quá trình huấn luyện mô hình với dữ liệu huấn luyện và val
trainer.fit(model, train_loader, val_loader)


In [ ]:
trainer.validate(model, test_loader)

#Bài tập

Hãy thêm def test_step vào class Model, tiến hành huấn luyện model với batch size là 256, lr là 5e-4, epochs là 50.

Form: https://forms.gle/FHVtVDXnjYBSBGEC6
